In [ ]:
import os
import numpy as np
import imageio
import cv2
from scipy import misc
from scipy import ndimage
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.image as mpimg

In [ ]:
def threshold(image, threshold):
    assert len(image.shape) == 2, "Must be grayscale image"
    thresh = np.zeros(image.shape)
    for rownum in range(len(image)):
        for colnum in range(len(image[rownum])):
            if image[rownum][colnum] > threshold:
                thresh[rownum][colnum] = 0
            else:
                thresh[rownum][colnum] = 255
    return thresh

def getHistGray(image):
    assert len(image.shape) == 2, "Must be grayscale image"
    hist = np.zeros(255)
    for row in image:
        for col in row:
            hist[int(col)] += 1
    return hist

def otsu(image):
    assert len(image.shape) == 2, "Must be grayscale image"
    th = _getOtsuThreshold(image)
    return threshold(image, th)


def _getOtsuThreshold(image):
    #import measure as m
    s = 0;
    histogram = getHistGray(image)
    for i in range(len(histogram)):
        s += i * histogram[i]
    sumB = 0
    wB = 0
    wF = 0
    mB = None
    mF = None
    m = 0.0
    between = 0.0
    threshold1 = 0.0
    threshold2 = 0.0
    total = len(image.ravel())
    for i in range(len(histogram)):
        wB += histogram[i]
        if wB == 0:
            continue;
        wF = total - wB
        if (wF == 0):
            break
        sumB += i * histogram[i]
        mB = sumB / wB
        mF = (s - sumB) / wF
        between = wB * wF * ((mB - mF) ** 2)
        if between >= m :
            threshold1 = i
            if between > m :
                threshold2 = i
            m = between
    return ( threshold1 + threshold2 ) / 2.0;


def np_binary_erosion(input_array, structure=np.ones((3,3)).astype(np.bool)):

    rows, cols = input_array.shape

    pad_shape = (
        input_array.shape[0] + structure.shape[0] - 1,
        input_array.shape[1] + structure.shape[1] - 1)
    input_pad_array = np.zeros(pad_shape).astype(np.bool)
    input_pad_array[1:rows+1,1:cols+1] = input_array
    binary_erosion = np.zeros(pad_shape).astype(np.bool)
    struc_mask = structure.astype(np.bool)
    for row in range(len(image)):
        for col in range(len(image[row])):
            binary_erosion[row+1,col+1] = np.min(
                input_pad_array[row:row+3, col:col+3][struc_mask])
    return binary_erosion[1:rows+1,1:cols+1]

def np_binary_dilation(img):
    out=np.array(img)
    h=img.shape[0]
    w=img.shape[1]
    D=np.zeros((h,w),dtype=int)
    for i in np.arange(2,h-2):
            for j in np.arange(2,w-2):
                a=img[i][j]
                if a.any():
                    out[i+1][j]=255
                    out[i][j+1]=255
    return out



def masking(image,mask):
    new_img=np.array(image)
    for row in range(len(image)):
        for col in range(len(image[row])):
            if mask[row,col]==0:
                new_img[row,col]=0
            if mask[row,col]==1:
                new_img[row,col]=image[row,col]
    return new_img

def convertToGreyScale(image):

    def getWeightedAvg(pixel):
        return 0.299*pixel[0] + 0.587*pixel[1] + 0.114*pixel[2]

    grey = np.zeros(image.shape[0:-1])
    for rownum in range(len(image)):
        for colnum in range(len(image[rownum])):
                grey[rownum][colnum] = getWeightedAvg(image[rownum][colnum])

    return grey

In [ ]:
image = imageio.imread('/content/drive/MyDrive/DATASET/benign/1 (56).jpg')
plt.title("Input image")
plt.imshow(image)
plt.show()

grey = convertToGreyScale(image)
plt.title("Greyscale image")
plt.imshow(grey,cmap=cm.Greys_r)
plt.show()

thresh = otsu(grey)
plt.title("Thresholded/Binary image")
plt.imshow(thresh, cmap = cm.Greys_r)
plt.show()

dilimg=ndimage.binary_dilation(thresh)
for x in range(25):
    dilimg=ndimage.binary_dilation(dilimg)
erimg=ndimage.binary_erosion(dilimg)
for x in range(25):
    erimg=ndimage.binary_erosion(erimg)
plt.title("opened image")
plt.imshow(erimg,cmap=cm.Greys_r)
plt.show()

imageio.imwrite('threshold.jpg', thresh)

new_img=masking(image,erimg)
plt.title("FINAL IMAGE")
plt.imshow(new_img)
plt.show()
imageio.imwrite('Result.jpg', new_img)


fig=plt.figure()
ax1=fig.add_subplot(2,2,1)
ax1.imshow(image)
ax2=fig.add_subplot(2,2,2)
ax2.imshow(grey,cmap=cm.Greys_r)
ax3=fig.add_subplot(2,2,3)
ax3.imshow(thresh,cmap=cm.Greys_r)
ax4=fig.add_subplot(2,2,4)
ax4.imshow(erimg,cmap=cm.Greys_r)
plt.show()

In [ ]:
import numpy as np
import math
import scipy
from scipy import misc
from scipy import ndimage
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import imageio

In [ ]:
def sobel(array):
        Gdash=array
        shape=array.shape
        gx=np.array([[-2,-1,0,1,2] for i in range(0,5)])
        gy=np.array([np.repeat(i,5) for i in range (-2,3)])
        gf=np.array([[np.exp(-(gx[i,j]*gx[i,j]+gy[i,j]*gy[i,j])/3) for j in range(0,5)] for i in range(0,5)])
        smooth=np.array([([np.sum(gf*array[i:i+5,j:j+5])/25 for j in range(0,shape[1]-4)]) for i in range(0,shape[0]-4)])
        for i in range(2,shape[0]-2):
            for j in range(2,shape[1]-2):
                 Gdash[i,j]=smooth[i-2,j-2]
        G=Gdash
        sobely=np.array([[3,0,-3],[10,0,-10],[3,0,-3]])
        sobelx=np.array([[3,10,3],[0,0,0],[-3,-10,-3]])
        Gx=np.array([([np.sum(sobelx*array[i:i+3,j:j+3])/9 for j in range(0,shape[1]-2)]) for i in range(0,shape[0]-2)])
        Gy=np.array([([np.sum(sobely*array[i:i+3,j:j+3])/9 for j in range(0,shape[1]-2)]) for i in range(0,shape[0]-2)])
        Gx=[[Gx[i,j]*Gx[i,j] for j in range(0,shape[1]-2)] for i in range(0,shape[0]-2)]
        Gy=[[Gy[i,j]*Gy[i,j] for j in range(0,shape[1]-2)] for i in range(0,shape[0]-2)]
        tempG=np.sqrt(np.add(Gx,Gy))
        for i in range(1,shape[0]-1):
            for j in range(1,shape[1]-1):
                 G[i,j]=tempG[i-1,j-1]
        G.astype(int)
        return G

def Area(img):
    naffect=0
    affect=0
    for rownum in range(len(img)):
        for colnum in range(len(img[rownum])):
            if img[rownum,colnum] == 0 :
                naffect=naffect+1
            else:
                affect=affect+1
    a=naffect-affect
    A=affect
    return a,A

def ColourCalc(hsvimg):
    mean=0
    SD=0
    for row in range(len(hsvimg)):
        for col in range(len(hsvimg[row])):
            mean=mean+hsvimg[row,col,0]
    for row in range(len(hsvimg)):
        for col in range(len(hsvimg[row])):
            SD=SD+(hsvimg[row,col,0]-mean)*(hsvimg[row,col,0]-mean)
    width,height,val=hsvimg.shape
    n=width*height
    SD=math.sqrt(SD)/(n-1)
    return SD

In [ ]:
if __name__== "__main__":

    img=imageio.imread('/content/drive/MyDrive/Result.jpg')
    img2=imageio.imread('/content/drive/MyDrive/threshold.jpg')
    plt.title('Processed image')
    plt.imshow(img)
    plt.show()
    dilimg=ndimage.binary_dilation(img2)
    for x in range(25):
        dilimg=ndimage.binary_dilation(dilimg)
    erimg=ndimage.binary_erosion(dilimg)
    for x in range(25):
        erimg=ndimage.binary_erosion(erimg)
    plt.title('Closed image')
    plt.imshow(erimg,cmap=cm.Greys_r)
    plt.show()
    a,A=Area(erimg)

    Asymmetry=((a/A)*100)/10
    if Asymmetry<0 :
        Asymmetry=Asymmetry*(-1)
    print('Asymmetry')
    print(Asymmetry)


    gigi=sobel(erimg)
    plt.imshow(gigi,cmap=cm.Greys_r)
    plt.show()
    pcount=0
    for row in range(len(gigi)):
        for col in range(len(gigi[row])):
            if gigi[row,col] == 1:
                pcount=pcount+1
    P=pcount
    Border=((P*P)/(4*3.14*A))/10
    print('Border')
    print(Border)

    n=(4*A)/P
    Diameter=(math.sqrt(n))/10
    print('Diameter')
    print(Diameter)

    hsvimg=matplotlib.colors.rgb_to_hsv(img)

    Colour=(ColourCalc(hsvimg))/10
    print('Colour')
    print(Colour)